# Response Evaluator

In [1]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [2]:
from llama_index import (
    TreeIndex,
    VectorStoreIndex,
    SimpleDirectoryReader,
    LLMPredictor,
    ServiceContext,
    Response,
)
from llama_index.llms import OpenAI
from llama_index.evaluation import ResponseEvaluator
import pandas as pd

pd.set_option("display.max_colwidth", 0)

INFO:numexpr.utils:Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
NumExpr defaulting to 8 threads.


In [3]:
# gpt-3 (davinci)
gpt3 = OpenAI(temperature=0, model="text-davinci-003")
service_context_gpt3 = ServiceContext.from_defaults(llm=gpt3)

# gpt-4
gpt4 = OpenAI(temperature=0, model="gpt-4")
service_context_gpt4 = ServiceContext.from_defaults(llm=gpt4)

In [4]:
evaluator = ResponseEvaluator(service_context=service_context_gpt3)
evaluator_gpt4 = ResponseEvaluator(service_context=service_context_gpt4)

In [15]:
documents = SimpleDirectoryReader("./test_wiki_data/").load_data()

In [17]:
# create tree index
tree_index = TreeIndex.from_documents(documents=documents)

INFO:llama_index.indices.common_tree.base:> Building index from nodes: 3 chunks
> Building index from nodes: 3 chunks


In [18]:
# create vector index
vector_index = VectorStoreIndex.from_documents(
    documents, service_context=ServiceContext.from_defaults(chunk_size=512)
)

In [32]:
# define jupyter display function
def display_eval_df(response: Response, eval_result: str) -> None:
    if response.source_nodes == []:
        print("no response!")
        return
    eval_df = pd.DataFrame(
        {
            "Response": str(response),
            "Source": response.source_nodes[0].node.text[:1000] + "...",
            "Evaluation Result": eval_result,
        },
        index=[0],
    )
    eval_df = eval_df.style.set_properties(
        **{
            "inline-size": "600px",
            "overflow-wrap": "break-word",
        },
        subset=["Response", "Source"]
    )
    display(eval_df)

In [20]:
query_engine = tree_index.as_query_engine()
response_tree = query_engine.query(
    "What battles took place in New York City in the American Revolution?"
)
eval_result = evaluator_gpt4.evaluate(response_tree)

INFO:llama_index.indices.tree.select_leaf_retriever:>[Level 0] Selected node: [1]/[1]
>[Level 0] Selected node: [1]/[1]
INFO:llama_index.indices.tree.select_leaf_retriever:>[Level 1] Selected node: [4]/[4]
>[Level 1] Selected node: [4]/[4]


In [27]:
display_eval_df(response_tree, eval_result)

,Response,Source,Evaluation Result
0,The Battle of Long Island and the Great Fire of New York.,"settlement was promptly renamed ""New York"" after the Duke of York (the future King James II and VII), who would eventually be deposed in the Glorious Revolution. After the founding, the duke gave part of the colony to proprietors George Carteret and John Berkeley. Fort Orange, 150 miles (240 km) north on the Hudson River, was renamed Albany after James's Scottish title. The transfer was confirmed in 1667 by the Treaty of Breda, which concluded the Second Anglo-Dutch War.On August 24, 1673, during the Third Anglo-Dutch War, Dutch captain Anthony Colve seized the colony of New York from the English at the behest of Cornelis Evertsen the Youngest and rechristened it ""New Orange"" after William III, the Prince of Orange. The Dutch would soon return the island to England under the Treaty of Westminster of November 1674.Several intertribal wars among the Native Americans and some epidemics brought on by contact with the Europeans caused sizeable population losses for the Lenape between the ye...",YES


In [28]:
query_engine = vector_index.as_query_engine()
response_vector = query_engine.query(
    "What battles took place in New York City in the American Revolution?"
)
eval_result = evaluator_gpt4.evaluate(response_vector)

In [29]:
display_eval_df(response_vector, eval_result)

,Response,Source,Evaluation Result
0,"The Battle of Long Island, which was the largest battle of the American Revolutionary War, took place in August 1776 within the modern-day borough of Brooklyn. The only attempt at a peaceful solution to the war took place at the Conference House on Staten Island between American delegates, including Benjamin Franklin, and British general Lord Howe on September 11, 1776.","enslaved few or several people. Others were hired out to work at labor. Slavery became integrally tied to New York's economy through the labor of slaves throughout the port, and the banking and shipping industries trading with the American South. During construction in Foley Square in the 1990s, the African Burying Ground was discovered; the cemetery included 10,000 to 20,000 of graves of colonial-era Africans, some enslaved and some free.The 1735 trial and acquittal in Manhattan of John Peter Zenger, who had been accused of seditious libel after criticizing colonial governor William Cosby, helped to establish the freedom of the press in North America. In 1754, Columbia University was founded under charter by King George II as King's College in Lower Manhattan. === American Revolution === The Stamp Act Congress met in New York in October 1765, as the Sons of Liberty organization emerged in the city and skirmished over the next ten years with British troops stationed there. The Battl...",YES


In [33]:
query_engine = tree_index.as_query_engine()
response_tree = query_engine.query("What are the airports in New York City?")
eval_result = evaluator_gpt4.evaluate(response_tree)

INFO:llama_index.indices.tree.select_leaf_retriever:>[Level 0] Selected node: [4]/[4]
>[Level 0] Selected node: [4]/[4]


In [34]:
display_eval_df(response_tree, eval_result)

no response!


In [37]:
query_engine = vector_index.as_query_engine()
response_vector = query_engine.query("What are the airports in New York City?")
eval_result = evaluator_gpt4.evaluate(response_vector)

In [38]:
display_eval_df(response_vector, eval_result)

,Response,Source,Evaluation Result
0,"The airports in New York City are John F. Kennedy International Airport, Newark Liberty International Airport, LaGuardia Airport, Long Island MacArthur Airport, Trenton–Mercer Airport, and Westchester County Airport.","busiest U.S. gateways for international air passengers, respectively, in 2012; as of 2011, JFK was the busiest airport for international passengers in North America.Plans have advanced to expand passenger volume at a fourth airport, Stewart International Airport near Newburgh, New York, by the Port Authority of New York and New Jersey. Plans were announced in July 2015 to entirely rebuild LaGuardia Airport in a multibillion-dollar project to replace its aging facilities. Other commercial airports in or serving the New York metropolitan area include Long Island MacArthur Airport, Trenton–Mercer Airport and Westchester County Airport. The primary general aviation airport serving the area is Teterboro Airport. === Ferries === The Staten Island Ferry is the world's busiest ferry route, carrying more than 23 million passengers from July 2015 through June 2016 on the 5.2-mile (8.4 km) route between Staten Island and Lower Manhattan and running 24 hours a day. Other ferry systems shuttle c...",YES


In [39]:
query_engine = vector_index.as_query_engine()
response_vector = query_engine.query("Who is the mayor of New York City?")
eval_result = evaluator_gpt4.evaluate(response_vector)

In [40]:
display_eval_df(response_vector, eval_result)

,Response,Source,Evaluation Result
0,"The mayor of New York City is Eric Adams. He was elected in 2021 with 67% of the vote, and assumed office on January 1, 2022.","Circuit and U.S. Court of International Trade are also based in New York, also on Foley Square in Manhattan. === Politics === The present mayor is Eric Adams. He was elected in 2021 with 67% of the vote, and assumed office on January 1, 2022. The Democratic Party holds the majority of public offices. As of April 2016, 69% of registered voters in the city are Democrats and 10% are Republicans. New York City has not been carried by a Republican presidential election since President Calvin Coolidge won the five boroughs in 1924. A Republican candidate for statewide office has not won all five boroughs of the city since it was incorporated in 1898. In 2012, Democrat Barack Obama became the first presidential candidate of any party to receive more than 80% of the overall vote in New York City, sweeping all five boroughs. Party platforms center on affordable housing, education, and economic development, and labor politics are of importance in the city. Thirteen out of 27 U.S. congressiona...",YES
